In [7]:

api_key='RGAPI-a11f7d8d-dbda-477a-8055-806cbff4dbae'
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import time
from urllib.error import HTTPError
# golbal variables
watcher = LolWatcher(api_key)
my_region = 'na1'


In [32]:
#get champ mapping
# check league's latest version
latest = watcher.data_dragon.versions_for_region('na1')['n']['champion']
#create dictionary of champions
static_champ_list = watcher.data_dragon.champions(latest, False, 'en_US')

champ_dict = {} 

for champ in static_champ_list['data'].keys(): # iterates thru champ names
   champ_dict[static_champ_list['data'][champ]['key']]=champ

#create dictionary of items

static_item_list = watcher.data_dragon.items(latest, False)

item_dict = {} 

for item in static_item_list['data'].keys(): # iterates thru item ids
   item_dict[item]=static_item_list['data'][item]['name']

static_spell_list = watcher.data_dragon.summoner_spells(latest, False)

spell_dict = {} 

for spell in static_spell_list['data'].keys(): # iterates thru item ids
   spell_dict[static_spell_list['data'][spell]['key']]=static_spell_list['data'][spell]['name']

# Queue mapping json . Couldn't Find in Data Dragon
queue_map=pd.read_json("queues.json")


In [38]:
#function to iterate thru all of my matches, and create a dataframe
def getMatchHistory(summonerName="", my_region='na1', max_matches=20):
    colnames=['minute', 'participantId', 'totalGold', 'level', 'xp', 'minionsKilled', 'jungleMinionsKilled',
              'gameId', 'queue', 'teamId', 'win', 'champion_name', 'role', 'lane', 'summonerName']
    Stats_DF=pd.DataFrame(columns=colnames) 
    me = watcher.summoner.by_name(my_region, summonerName)
    my_matches = watcher.match.matchlist_by_account(my_region, me['accountId'])
    print(summonerName)
    num_matches=len(my_matches['matches'])
    num_to_run=min(num_matches, max_matches)
    print("num_matches to run: ", num_to_run)

    #for last_match in my_matches['matches']:
    for i in range(num_to_run):
        last_match=my_matches['matches'][i]
        gameId=last_match['gameId']

        print("gameId: ", gameId, "runcount: ", i)
        #limited to 100 requests every 2 minutes. 33 matches is 100 requests, but i do 33 matches in about 1 minute. add this code to slow down my program.
        #print(i%24, (i%24==0 and i>0))
        #if(i%24==0 and i>0):
        #    print('pausing')
        #    time.sleep(150)
        #    print('unpaused')
        
        #try to prevent bad gateway
        match_detail_fail=True
        while(match_detail_fail == True):
            try:
                match_detail = watcher.match.by_id(my_region, gameId)
                match_detail_fail = False
            except :
                print("HTTP Error, let's try again")
            
        #get queue, convert to description from ID number
        #['5v5 ARAM games', '5v5 Draft Pick games', '5v5 Ranked Flex games', '5v5 Ranked Solo games', 'Clash games', 'URF games']
        queue=last_match['queue']
        queue_map1=queue_map.loc[queue_map['queueId'] == queue]
        queue_desc=list(queue_map1['description'])[0]
        #get match data BIG stats
        participants = []
        for row in match_detail['participants']:
            #print(row.keys())
            participants_row = {}
            participants_row['participantId'] = row['participantId']
            participants_row['gameId'] = gameId
            participants_row['queue'] = queue_desc
            participants_row['champion'] = row['championId']
            participants_row['spl0'] = row['spell1Id']
            participants_row['spl1'] = row['spell2Id']
            participants_row['win'] = row['stats']['win']
            participants_row['kills'] = row['stats']['kills']
            participants_row['deaths'] = row['stats']['deaths']
            participants_row['assists'] = row['stats']['assists']
            participants_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
            participants_row['goldEarned'] = row['stats']['goldEarned']
            participants_row['champLevel'] = row['stats']['champLevel']
            participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
            if row['teamId'] == 100:
                participants_row['teamId'] = "Blue"
            else:
                participants_row['teamId'] = "Red"
            participants_row['win'] = row['stats']['win']
            participants_row['role'] = row['timeline']['role']
            participants_row['lane'] = row['timeline']['lane']
            participants_row['it0'] = row['stats']['item0']
            participants_row['it1'] = row['stats']['item1']
            participants_row['it2'] = row['stats']['item2']
            participants_row['it3'] = row['stats']['item3']
            participants_row['it4'] = row['stats']['item4']
            participants_row['it5'] = row['stats']['item5']
            participants.append(participants_row)
        match_details_df = pd.DataFrame(participants)
        match_details_df['champion_name']=match_details_df['champion'].apply(str).map(champ_dict)
        match_details_df['spell0']=match_details_df['spl0'].apply(str).map(spell_dict)
        match_details_df['spell1']=match_details_df['spl1'].apply(str).map(spell_dict)
        match_details_df['item0']=match_details_df['it0'].apply(str).map(item_dict)
        match_details_df['item1']=match_details_df['it1'].apply(str).map(item_dict)
        match_details_df['item2']=match_details_df['it2'].apply(str).map(item_dict)
        match_details_df['item3']=match_details_df['it3'].apply(str).map(item_dict)
        match_details_df['item4']=match_details_df['it4'].apply(str).map(item_dict)
        match_details_df['item5']=match_details_df['it5'].apply(str).map(item_dict)

        match_details_df2=match_details_df[['gameId','participantId','queue','teamId','win','champion_name','role','lane','spell0','spell1','item0','item1','item2','item3','item4','item5']].copy()

        match_detail_fail=True
        while(match_detail_fail == True):
            try:
                match_summoners = watcher.match.by_id(my_region, gameId)
                match_detail_fail = False
            except :
                print("HTTP Error, let's try again")

        sumnames_index=['participantId','summonerName']
        summmonersNamesDF=pd.DataFrame(columns=sumnames_index)
        for i in range(10):
            x=[(match_summoners['participantIdentities'][i]['participantId']),
                        match_summoners['participantIdentities'][i]['player']['summonerName']]
            sum_names_series = pd.Series(x, index = sumnames_index)
            summmonersNamesDF=summmonersNamesDF.append(sum_names_series,ignore_index="True")
        summmonersNamesDF
        
        #get match data by minute

        match_detail_fail=True
        while(match_detail_fail == True):
            try:
                match_timeline = watcher.match.timeline_by_match(my_region, gameId)
                match_detail_fail = False
            except :
                print("HTTP Error, let's try again")

        l=len(match_timeline['frames'])
        data_fields=['participantId','totalGold','level','xp','minionsKilled','jungleMinionsKilled','pos_x','pos_y']
        golddf=pd.DataFrame(columns=['minute']+data_fields)

        for i in range(l): #loops thru minutes
            

            for j in match_timeline['frames'][i]['participantFrames']:#loops thru champions
                pos_x=999999
                pos_y=999999
                #check if position data is available
                if 'position' in match_timeline['frames'][i]['participantFrames'][j]:
                    pos_x=match_timeline['frames'][i]['participantFrames'][j]['position']['x']
                    pos_y=match_timeline['frames'][i]['participantFrames'][j]['position']['y']

                champ_data=[i,  match_timeline['frames'][i]['participantFrames'][j][data_fields[0]],
                                match_timeline['frames'][i]['participantFrames'][j][data_fields[1]],
                                match_timeline['frames'][i]['participantFrames'][j][data_fields[2]],
                                match_timeline['frames'][i]['participantFrames'][j][data_fields[3]],
                                match_timeline['frames'][i]['participantFrames'][j][data_fields[4]],
                                match_timeline['frames'][i]['participantFrames'][j][data_fields[5]],
                                pos_x,
                                pos_y]
            
                champ_data_series = pd.Series(champ_data, index = golddf.columns)
                #print(champ_data_series)
                golddf=golddf.append(champ_data_series,ignore_index="True")

        merged_1 = golddf.merge(match_details_df2,how='left',on='participantId')
        merged_2 = merged_1.merge(summmonersNamesDF,how='left',on='participantId')

        merged_3=merged_2.sort_values(["champion_name",'minute'])
        Stats_DF=Stats_DF.append(merged_3)
    return Stats_DF



In [42]:
beg=time.perf_counter()
m=100
testStatsDF=getMatchHistory(summonerName="Souporsecret", max_matches=m)

end=time.perf_counter()
runtime=end-beg
runtime_mins=int(runtime/60)
runtime_secs=runtime%60
print(runtime_mins, " minutes and ", runtime_secs, "seconds. ", "\nSeconds per iteration: ", runtime/m)

Souporsecret
num_matches to run:  100
gameId:  3849064472 runcount:  0
gameId:  3849016581 runcount:  1
gameId:  3847601269 runcount:  2
gameId:  3847507333 runcount:  3
gameId:  3844648489 runcount:  4
gameId:  3844663230 runcount:  5
gameId:  3844649862 runcount:  6
gameId:  3844175436 runcount:  7
gameId:  3844151451 runcount:  8
gameId:  3844038154 runcount:  9
gameId:  3844020687 runcount:  10
gameId:  3843896927 runcount:  11
gameId:  3843322544 runcount:  12
gameId:  3843276433 runcount:  13
gameId:  3843193543 runcount:  14
gameId:  3842758429 runcount:  15
gameId:  3842727812 runcount:  16
gameId:  3842743671 runcount:  17
gameId:  3842657060 runcount:  18
gameId:  3840383384 runcount:  19
gameId:  3836522152 runcount:  20
gameId:  3836474151 runcount:  21
gameId:  3835675132 runcount:  22
gameId:  3835063919 runcount:  23
gameId:  3835020392 runcount:  24
gameId:  3834992235 runcount:  25
gameId:  3834857956 runcount:  26
gameId:  3834810362 runcount:  27
gameId:  3834718751 

In [43]:
filename=str(m)+"_Match_Stats_Souporsecretv4(spell).csv"
testStatsDF.to_csv(filename,index=False)
pd.options.display.max_columns = None
testStatsDF.tail(8)

,minute,participantId,totalGold,level,xp,minionsKilled,jungleMinionsKilled,gameId,queue,teamId,win,champion_name,role,lane,summonerName,pos_x,pos_y,spell0,spell1,item0,item1,item2,item3,item4,item5
348,34,10,14838,15,14512,215,38,3765756183,5v5 Draft Pick games,Red,True,Tristana,DUO_CARRY,BOTTOM,AerialBlast,6483,7876,Heal,Flash,Galeforce,Mercurial Scimitar,Stormrazor,Mortal Reminder,Berserker's Greaves,Infinity Edge
358,35,10,15360,16,15352,217,38,3765756183,5v5 Draft Pick games,Red,True,Tristana,DUO_CARRY,BOTTOM,AerialBlast,10466,13851,Heal,Flash,Galeforce,Mercurial Scimitar,Stormrazor,Mortal Reminder,Berserker's Greaves,Infinity Edge
368,36,10,15817,16,16004,226,40,3765756183,5v5 Draft Pick games,Red,True,Tristana,DUO_CARRY,BOTTOM,AerialBlast,11027,8692,Heal,Flash,Galeforce,Mercurial Scimitar,Stormrazor,Mortal Reminder,Berserker's Greaves,Infinity Edge
378,37,10,16617,17,17199,243,47,3765756183,5v5 Draft Pick games,Red,True,Tristana,DUO_CARRY,BOTTOM,AerialBlast,7674,7291,Heal,Flash,Galeforce,Mercurial Scimitar,Stormrazor,Mortal Reminder,Berserker's Greaves,Infinity Edge
388,38,10,17140,17,17914,254,47,3765756183,5v5 Draft Pick games,Red,True,Tristana,DUO_CARRY,BOTTOM,AerialBlast,9433,2409,Heal,Flash,Galeforce,Mercurial Scimitar,Stormrazor,Mortal Reminder,Berserker's Greaves,Infinity Edge
398,39,10,17367,17,18354,254,51,3765756183,5v5 Draft Pick games,Red,True,Tristana,DUO_CARRY,BOTTOM,AerialBlast,10051,10623,Heal,Flash,Galeforce,Mercurial Scimitar,Stormrazor,Mortal Reminder,Berserker's Greaves,Infinity Edge
408,40,10,18353,18,19503,267,51,3765756183,5v5 Draft Pick games,Red,True,Tristana,DUO_CARRY,BOTTOM,AerialBlast,4456,4269,Heal,Flash,Galeforce,Mercurial Scimitar,Stormrazor,Mortal Reminder,Berserker's Greaves,Infinity Edge
418,41,10,18785,18,19764,269,51,3765756183,5v5 Draft Pick games,Red,True,Tristana,DUO_CARRY,BOTTOM,AerialBlast,999999,999999,Heal,Flash,Galeforce,Mercurial Scimitar,Stormrazor,Mortal Reminder,Berserker's Greaves,Infinity Edge
